# DCGAN
Para este notebook se van a crear dos DCGAN, una entrenada con imágenes benignas
y otra DCGAN con imágenes malignas.

In [ ]:
%pip install -r requirements.txt --quiet
%pip install -r dev-requirements.txt --quiet

In [1]:
import file_utils

In [2]:
path = "./BreaKHis_v1/histology_slides/"
images_path = file_utils.get_image_path(path)

In [5]:
from torchvision import datasets, transforms
from breakhis import BreastHistology

image_size = 64
ngpu = 1

data_transform = transforms.Compose([
    # Ya que hay algunas imagenes que tienen menor resolución
    transforms.Resize(image_size),
    transforms.CenterCrop(image_size),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
])

# 
Solo correr la siguiente celda en caso de querer generar las imágenes en 
un direcotrio nuevo, esto para posteriormente generar las estádisticas necesarias
para calcular el FID.

In [ ]:
import torchvision.utils as vutils
import os

save_image_transform = transforms.Compose([
    # Ya que hay algunas imagenes que tienen menor resolución
    transforms.Resize(image_size),
    transforms.ToTensor(),
])

breast_histology_slides = BreastHistology(root_dir=path,
                                          transform=save_image_transform)

os.makedirs("original_images", exist_ok=True)

for index, image in enumerate(iter(breast_histology_slides)):
    filename = f"image_{index}.png"
    vutils.save_image(image[0], os.path.join("original_images", filename))

# Calcular FID score e IS
A continuación se calculan ambas métricas. Se supone que ambas métricas ayudan a calificar
el desempeño que tiene una GAN. [paper](https://arxiv.org/abs/1911.07023)

## FID Score


## Inception Score